In [1]:
%load_ext autoreload
%autoreload 2

## Llama 8b

In [2]:
import os
while 'notebooks' in os.getcwd():
    os.chdir("..")

import numpy as np
import pandas as pd 
from src.utils import train_test_split
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
from src.preprocessing import TextDataset
import torch
from torch.utils.data import DataLoader, Dataset
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, LoggingHandler
import logging
from copy import deepcopy
from sklearn.decomposition import PCA
from huggingface_hub import notebook_login
from sklearn.ensemble import RandomForestClassifier
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.utils.class_weight import compute_class_weight
from collections import defaultdict
from transformers import BitsAndBytesConfig


tqdm.pandas()

In [16]:
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# # Tokenizer and model setup
# 

# base_model = AutoModelForSequenceClassification.from_pretrained(
#     "meta-llama/Meta-Llama-3-8B-Instruct",
#     quantization_config=quantization_config,
#     device_map="auto",
#     attn_implementation="eager",
#     cache_dir="/Data"
# )

base_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    cache_dir="/Data"
)

tokenizer = AutoTokenizer.from_pretrained(
    "bert-base-uncased",
    cache_dir="/Data"
)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/users/eleves-a/2022/pedro.silva/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
base_path = "data"
all_df = []
for file in os.listdir(base_path):

    temp_df = pd.read_csv(os.path.join(base_path, file), index_col= 0 )
    all_df.append(temp_df)

train_df = pd.concat(all_df)
train_df = train_df.drop_duplicates("originalTweet")

mapper = defaultdict(lambda : 1)
mapper['no'] = 0

train_df['label'] = train_df['event'].map(mapper)

train_df.loc[(train_df['EventType'] == 0) & (train_df['label'] == 1), 'label'] = 0

train_df = train_df.sample(frac = 1., replace=False)\
    .dropna()

test_matches = list(np.random.choice(train_df.MatchID.unique(), size=4))

test_df = train_df.query(f"MatchID in  {test_matches}")
train_df = train_df.query(f"MatchID not in {test_matches}")

In [19]:
device = 'cuda'

labels = train_df["EventType"].tolist()
# class_weights = compute_class_weights(labels).to(device)

base_model.to(device)
# Define weighted loss function
loss_fn = torch.nn.CrossEntropyLoss()
# for param in model.model.parameters():
#     param.requires_grad = False

# lora_config = LoraConfig(
#     task_type=TaskType.SEQ_CLS,  # Sequence classification task
#     r=16,  # LoRA rank
#     lora_alpha=16,  # Scaling factor
#     lora_dropout=0.1  # Dropout for LoRA layers
# )
# model = get_peft_model(base_model, lora_config).to(device)
    
optimizer = torch.optim.AdamW(base_model.parameters(), lr=5e-5)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", cache_dir = '/Da/ta')
n_epochs = 5

for epoch in range(n_epochs):
    for idx, row in tqdm(train_df.iterrows(), total = len(train_df)):
        texts = "Is the following text represents a football event such as goal, foul, halftime, full time etc?" + "\n" + row['originalTweet']
        max_probs = []

        tokens = tokenizer(texts, return_tensors='pt')

        label = torch.tensor(row['label']).to(device)

        input_ids = tokens["input_ids"].to(device)
        attention_mask = tokens["attention_mask"].to(device)

        with torch.autocast( device_type = 'cuda'):
            outputs = base_model(input_ids=input_ids, attention_mask=attention_mask)

        # Compute loss and backpropagate
        loss = loss_fn(outputs.logits, label.to(torch.long).unsqueeze(-1))
            

        loss.backward()

        
        optimizer.step()
        optimizer.zero_grad()

    y_true_val =[]
    y_pred_val = []

    y_true_val =[]
    y_pred_val = []

    with torch.no_grad():
        for idx, row in tqdm(test_df.iterrows(), total = len(test_df)):
            texts = "Is the following text represents a football event such as goal, foul, halftime, full time etc?" + "\n" + row['originalTweet']
        
            tokens = tokenizer(texts, return_tensors='pt')


            label = torch.tensor(row['label']).to(device)

            input_ids = tokens["input_ids"].to(device)
            attention_mask = tokens["attention_mask"].to(device)
            # count = torch.ones(len(texts)).to(device).unsqueeze(dim = -1) * row['ID']

            with torch.autocast( device_type = 'cuda'):
                outputs = base_model(input_ids=input_ids, attention_mask=attention_mask)

            p = torch.argmax(outputs.logits, dim = -1)

            y_true_val.append(row['label'])
            y_pred_val.append(p.item())

    acc_val = accuracy_score(y_true_val, y_pred_val)
    print(acc_val)

  1%|▏         | 919/68488 [00:31<38:22, 29.34it/s]


KeyboardInterrupt: 

In [21]:
y_true_val =[]
y_pred_val = []

with torch.no_grad():
    for idx, row in tqdm(test_df.iterrows(), total = len(test_df)):
        texts = "Is the following text represents a football event such as goal, foul, halftime, full time etc?" + "\n" + row['originalTweet']

        tokens = tokenizer(texts, return_tensors='pt')


        label = torch.tensor(row['label']).to(device)

        input_ids = tokens["input_ids"].to(device)
        attention_mask = tokens["attention_mask"].to(device)
        # count = torch.ones(len(texts)).to(device).unsqueeze(dim = -1) * row['ID']

        with torch.autocast( device_type = 'cuda'):
            outputs = base_model(input_ids=input_ids, attention_mask=attention_mask)

        p = torch.argmax(outputs.logits, dim = -1)

        y_true_val.append(row['label'])
        y_pred_val.append(p.item())

acc_val = accuracy_score(y_true_val, y_pred_val)
print(acc_val)

  0%|          | 0/12124 [00:00<?, ?it/s]

 20%|█▉        | 2422/12124 [00:18<01:12, 133.22it/s]


KeyboardInterrupt: 

In [23]:
confusion_matrix(y_true_val, y_pred_val)

array([[1909,    0],
       [ 513,    0]])

In [9]:
row['label']

0